In [7]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, precision_score, recall_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [9]:
df = pd.read_csv('landmarks_dataset_balanced__M_A_W_H.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
X = df.drop('label', axis=1).values
y = df['label'].values


In [17]:
print('total label :', len(y))
print('contoh label :', y)
print('total vektor :', len(X))
print('contoh vektor :', X)

total label : 5854
contoh label : [1 0 1 ... 1 1 0]
total vektor : 5854
contoh vektor : [[398 334 407 ... 215 612 190]
 [326 386 319 ... 270 627 202]
 [320 399 324 ... 283 638 212]
 ...
 [349 372 341 ... 260 611 204]
 [336 383 331 ... 251 648 183]
 [307 372 316 ... 237 635 244]]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'weights': ['uniform', 'distance']
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['liblinear', 'saga']
}

# Models
svc = SVC()
knn = KNeighborsClassifier()
rf = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
lr = LogisticRegression(random_state=42)

models = {
    'SVC': (svc, param_grid_svc),
    'KNN': (knn, param_grid_knn),
    'RandomForest': (rf, param_grid_rf),
    'XGBoost': (xgb, param_grid_xgb),
    'LogisticRegression': (lr, param_grid_lr)
}


In [6]:
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted')
}
start_time = time.time()

results = []
best_models = {}


for model_name, (model, param_grid) in models.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, scoring=scoring, refit='accuracy', return_train_score=True)
    grid_search.fit(X_train, y_train)

    best_models[model_name] = grid_search.best_estimator_

    num_splits = len([key for key in grid_search.cv_results_.keys() if key.startswith('split') and key.endswith('_test_accuracy')])


    for i in range(len(grid_search.cv_results_['mean_test_accuracy'])):
        params = grid_search.cv_results_['params'][i]
        mean_accuracy = grid_search.cv_results_['mean_test_accuracy'][i]
        mean_precision = grid_search.cv_results_['mean_test_precision'][i]
        mean_recall = grid_search.cv_results_['mean_test_recall'][i]
        mean_fit_time = grid_search.cv_results_['mean_fit_time'][i]
        mean_score_time = grid_search.cv_results_['mean_score_time'][i]
        total_training_time = mean_fit_time + mean_score_time


        split_scores = {f'split{split_idx}_test_accuracy': grid_search.cv_results_[f'split{split_idx}_test_accuracy'][i] for split_idx in range(num_splits)}
        split_precisions = {f'split{split_idx}_test_precision': grid_search.cv_results_[f'split{split_idx}_test_precision'][i] for split_idx in range(num_splits)}
        split_recalls = {f'split{split_idx}_test_recall': grid_search.cv_results_[f'split{split_idx}_test_recall'][i] for split_idx in range(num_splits)}

        result = {
            'model': model_name,
            'params': params,
            'mean_test_accuracy': mean_accuracy,
            'mean_test_precision': mean_precision,
            'mean_test_recall': mean_recall,
            'mean_fit_time': mean_fit_time,
            'mean_score_time': mean_score_time,
            'total_training_time': total_training_time
        }
        result.update(split_scores)
        result.update(split_precisions)
        result.update(split_recalls)

        results.append(result)


    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best cross-validation accuracy for {model_name}: {grid_search.best_score_}")


results_df = pd.DataFrame(results)
results_df.to_excel('/content/drive/MyDrive/dataset/Excels/NSplit/Notebooks/M_A_W_H/grid_search_results_M_A_W_H.xlsx', index=False)


elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time} seconds")
print('GColabs Traning Session')

Best parameters for SVC: {'C': 10, 'kernel': 'poly'}
Best cross-validation accuracy for SVC: 0.8054648916589215
Best parameters for KNN: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Best cross-validation accuracy for KNN: 0.8819159696025368
Best parameters for RandomForest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation accuracy for RandomForest: 0.8861840067063949
Best parameters for XGBoost: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 300}
Best cross-validation accuracy for XGBoost: 0.8968591110382155


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Best parameters for LogisticRegression: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best cross-validation accuracy for LogisticRegression: 0.7772793541450258
Elapsed time: 15797.172466039658 seconds
GColabs Traning Session
